# L9c: Skip-Gram Model for Word Embeddings
In this lecture, we explore the Skip-Gram model, a popular method for learning word embeddings by predicting context words from a target word. This approach is complementary to the Continuous Bag of Words (CBOW) model discussed earlier.

> __Learning Objectives:__
> 
> Three learning objectives go here.

Let's get started!
___

## Summary
One concise, direct summary sentence goes here.

> __Key Takeaways:__
>
> Three key takeaways go here.

One concise, direct concluding sentence goes here.
___